In [59]:
import numpy as np
import pandas as pd

In [60]:
#importing the dataframes
books=pd.read_csv('Books.csv')
ratings=pd.read_csv('Ratings.csv')
users=pd.read_csv('Users.csv')

C:\Users\SHIVAM\AppData\Local\Temp\ipykernel_221424\1325093156.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv('Books.csv')


In [61]:
users.shape

(278858, 3)

In [62]:
ratings_with_name = ratings.merge(books,on='ISBN')   #merging on basis of ratings and books

#Find the avg votes on the books and no of votes must be greater than 250

# new_df['User-ID'].value_counts()
# new_df['Book-Title'].value_counts().shape

In [63]:
#classifying the df on the basis of groupby on book title
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()


#renaming the cols
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)

In [64]:

#to calculate mean values of the ratings
avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()


#renaming the cols to mean_ratings
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'},inplace=True)

C:\Users\SHIVAM\AppData\Local\Temp\ipykernel_221424\3781772380.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()


In [65]:
# Convert 'Book-Rating' column to numeric (assuming it contains integers)
ratings_with_name['Book-Rating'] = pd.to_numeric(ratings_with_name['Book-Rating'], errors='coerce')

# Calculate mean ratings
avg_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'}, inplace=True)


In [66]:
#merging the num ratings and mean ratings
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')


#sorting the values based on ratings
# i,e, num votes>50
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)


In [67]:
#merging the num and mean wala df and book on bookk title
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]

popular_df['Book-Title']


0      Harry Potter and the Prisoner of Azkaban (Book 3)
3           Harry Potter and the Goblet of Fire (Book 4)
5         Harry Potter and the Sorcerer's Stone (Book 1)
9      Harry Potter and the Order of the Phoenix (Boo...
13      Harry Potter and the Chamber of Secrets (Book 2)
16     The Hobbit : The Enchanting Prelude to The Lor...
17     The Fellowship of the Ring (The Lord of the Ri...
26     Harry Potter and the Sorcerer's Stone (Harry P...
28        The Two Towers (The Lord of the Rings, Part 2)
39                                 To Kill a Mockingbird
47                                     The Da Vinci Code
53                    The Five People You Meet in Heaven
55                                The Catcher in the Rye
62                             The Lovely Bones: A Novel
63                                                  1984
72                              Prodigal Summer: A Novel
73                                            Neverwhere
78                             

COLLABRATIVE FILTERRING BASED PROGRAM


In [68]:
ratings_with_names = pd.merge(ratings,books,on='ISBN')
# ratings_with_names
# ratings

In [69]:
#Applying groupby to fetch out users having voted > 200
x=ratings_with_names.groupby('User-ID').count()['Book-Rating']>200
geeks=x[x].index
filtered_ratings=ratings_with_names[ratings_with_names['User-ID'].isin(geeks)]
#Filtered ratings having more than 200 books

In [70]:
#Also filtering the books which have at least gotten more than 50 votes
y=filtered_ratings.groupby('Book-Title').count()['Book-Rating']>=50
rated_books=y[y].index
famous_books=filtered_ratings[filtered_ratings['Book-Title'].isin(rated_books)]   #Famous book which has gotten more than 50 votes

#Creating pivot table rows consisting of filtered df and columns consiting of users
pt=famous_books.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
pt.fillna(0,inplace=True) #Filling out the missing values
book_list=famous_books['Book-Title'].unique()

In [71]:
from sklearn.metrics.pairwise import cosine_similarity #importing library

#Finding out the similarity score between each books among them
similarity_scores = cosine_similarity(pt)

In [72]:
#Defining a function

def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]

    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))

        data.append(item)

    return data

In [80]:
pickle.dump(pt,open('pt.pkl','wb'))

In [81]:
pickle.dump(books,open('books.pkl','wb'))

In [82]:
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))

In [21]:
recommend('The Bean Trees')

[['Animal Dreams',
  'Barbara Kingsolver',
  'http://images.amazon.com/images/P/0060921145.01.MZZZZZZZ.jpg'],
 ['Bel Canto: A Novel',
  'Ann Patchett',
  'http://images.amazon.com/images/P/0060934417.01.MZZZZZZZ.jpg'],
 ["She's Come Undone (Oprah's Book Club (Paperback))",
  'Wally Lamb',
  'http://images.amazon.com/images/P/0671003755.01.MZZZZZZZ.jpg'],
 ['Pigs in Heaven',
  'Barbara Kingsolver',
  'http://images.amazon.com/images/P/0060168013.01.MZZZZZZZ.jpg']]

In [45]:
import pickle

In [47]:
pickle.dump(book_list,open('Boooks.pkl','wb'))

In [52]:
tuple(book_list)

('The Notebook',
 'A Painted House',
 'Lightning',
 'Dark Paradise',
 'Night Sins',
 'How Stella Got Her Groove Back',
 'Waiting to Exhale',
 'The Pillars of the Earth',
 'The Dark Half',
 'Harry Potter and the Order of the Phoenix (Book 5)',
 'False Memory',
 "Bridget Jones's Diary",
 'A Wrinkle In Time',
 'The Boy Next Door',
 'Artemis Fowl (Artemis Fowl, Book 1)',
 'The Last Time They Met : A Novel',
 'Along Came a Spider (Alex Cross Novels)',
 'Whispers',
 "Ender's Game (Ender Wiggins Saga (Paperback))",
 'The Jester',
 'The King of Torts',
 'Politically Correct Bedtime Stories: Modern Tales for Our Life and Times',
 'Matilda',
 'The Lovely Bones: A Novel',
 'The Da Vinci Code',
 'The Joy Luck Club',
 'Wild Animus',
 'Coraline',
 'Harry Potter and the Chamber of Secrets (Book 2)',
 "Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
 'To Kill a Mockingbird',
 'Four To Score (A Stephanie Plum Novel)',
 'Southern Cross',
 'The Chamber',
 "The Idiot Girls' Action Adven